In [1]:
%%capture
import warnings
from datetime import date, timedelta
from google.colab import drive
!pip install seaborn
!pip install yahoo_fin
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import holidays
import random
from google.colab import files
from statistics import mode
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import concurrent.futures
!pip install gdown
import gdown
from scipy.stats import zscore
drive.mount('/content/drive')
import yahoo_fin.stock_info as si
# Non mostro la numerazione scientifica
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Mostro separatori delle migliai con la virgola
pd.options.display.float_format = '{:,.3f}'.format

cartella_csv = '/content/drive/MyDrive/Investimenti/finanza quant/'


In [10]:

# Debugging cell (facoltativo)
import sys
import pandas as pd
print("Python version:", sys.version)
print("Pandas version:", pd.__version__)


Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Pandas version: 2.0.3


In [5]:
# Solo i tickers che ho in portafoglio

prtf = ['TRMD', 'MSFT', 'STLA', 'NVDA', 'MC.PA', 'LRCX', 'META', 'AMAT', 'CFRUY', 'URI', 'AAPL', 'DHI', 'CHRD', 'LPG'
,'SMCI','MOD']

symbol_list = prtf

all_tickers_filtered = prtf

all_tickers = prtf

print(len(prtf))

16


In [ ]:
# # 6500 ticker ci mette un paio di minuti
# def fetch_fundamental_data(symbol):
#     try:
#         stock = yf.Ticker(symbol).info
#         return stock
#     except Exception as e:
#         print(f"Ignorato errore per il simbolo {symbol}: {e}")
#         return None

# # Specify the maximum number of threads (adjust according to your needs)
# max_threads = 10

# # Utilizza tqdm per aggiungere una progress bar al ciclo
# with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
#     # Use executor.map to parallelize the fetching of data
#     fundamental_data = list(tqdm(executor.map(fetch_fundamental_data, all_tickers), total=len(all_tickers), desc="Fetching data"))

# # Remove None values (symbols that had errors)
# fundamental_data = [data for data in fundamental_data if data is not None]

# # Visualizza il DataFrame finale
# df_fundamental = pd.DataFrame(fundamental_data)


Fetching data: 100%|██████████| 16/16 [00:02<00:00,  6.81it/s]


In [9]:

def fetch_fundamental_data(symbol):
    try:
        stock = yf.Ticker(symbol).info
        return stock
    except Exception as e:
        print(f"Ignorato errore per il simbolo {symbol}: {e}")
        return None

# Inizializza una lista per conservare i dati fondamentali
fundamental_data = []

# Utilizza tqdm per aggiungere una progress bar al ciclo
# for symbol in tqdm(all_tickers, desc="Fetching data"):
#     data = fetch_fundamental_data(symbol)
#     if data is not None:
#         fundamental_data.append(data)

# Crea il DataFrame finale dai dati fondamentali raccolti
df_fundamental = pd.DataFrame(fundamental_data)

In [8]:
# df_filtered = df_fundamental[(df_fundamental['industry'] == 'Oil & Gas Midstream') & (df_fundamental['marketCap'] > 3000000000)][['shortName', 'marketCap', 'dividendYield']].assign(dividendYield=lambda x: x['dividendYield'] * 100).sort_values(by='dividendYield', ascending=False)
# df_filtered = df_fundamental[(df_fundamental['dividendYield'] > 0.05) & (df_fundamental['marketCap'] > 3000000000)  ][['shortName', 'marketCap', 'dividendYield', 'country']].assign(dividendYield=lambda x: x['dividendYield'] * 100).sort_values(by='dividendYield', ascending=False)

# Definisci la funzione che esegue la moltiplicazione
def multiply_columns(df):
    # df['dividendYield'] *= 1
    # df['ebitdaMargins'] *= 1
    # df['returnOnEquity'] *= 1
    #df['priceToHighRatio'] = df['currentPrice'] / df['fiftyTwoWeekHigh'] * 100
    df['mostRecentQuarter'] = pd.to_datetime(df['mostRecentQuarter'], unit='s').dt.date
    return df.drop(columns=['currentPrice', 'fiftyTwoWeekHigh'])  #non mi interessa portarmi dietro df['currentPrice'] / df['fiftyTwoWeekHigh']


# Filtra e seleziona il DataFrame, applica la funzione e ordina
df_filtered = (
    df_fundamental
    #  [
        #(df_fundamental['industry'] == 'Oil & Gas Midstream') &
    # (df_fundamental['marketCap'] > 2000000000)]
    [['shortName',  'industry','dividendYield', 'ebitdaMargins', 'recommendationKey','totalCashPerShare',
      'returnOnEquity', 'currentPrice', 'fiftyTwoWeekHigh','earningsQuarterlyGrowth','mostRecentQuarter'
      #'priceToHighRatio'
      ]]
    .pipe(multiply_columns)
    .sort_values(by='dividendYield', ascending=False)
)

#df_filtered = df_fundamental[(df_fundamental['industry'] == 'Oil & Gas Midstream') & (df_fundamental['marketCap'] > 2000000000)][['shortName', 'marketCap','52WeekChange','dividendYield','ebitdaMargins','recommendationKey']].assign(dividendYield=lambda x: x[['dividendYield','ebitdaMargins']] * 100).sort_values(by='dividendYield', ascending=False)
df_filtered
nuovo = df_filtered
nuovo

<ipython-input-8-660a691eccd3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mostRecentQuarter'] = pd.to_datetime(df['mostRecentQuarter'], unit='s').dt.date


,shortName,industry,dividendYield,ebitdaMargins,recommendationKey,totalCashPerShare,returnOnEquity,earningsQuarterlyGrowth,mostRecentQuarter
0,TORM plc,Oil & Gas Midstream,0.154,0.537,strong_buy,6.307,0.378,0.365,2024-03-31
13,Dorian LPG Ltd.,Oil & Gas Midstream,0.083,0.701,buy,7.239,0.324,0.042,2024-03-31
2,Stellantis N.V.,Auto Manufacturers,0.074,0.146,buy,12.124,0.241,-0.132,2023-12-31
12,Chord Energy Corporation,Oil & Gas E&P,0.056,0.482,buy,7.108,0.188,-0.329,2024-03-31
8,COMPAGNIE FINANCIERE RICHEMONT,Luxury Goods,0.018,0.266,hold,33.021,0.193,-0.205,2024-03-31
4,LVMH,Luxury Goods,0.017,0.293,buy,22.596,0.267,-0.114,2023-12-31
9,"United Rentals, Inc.",Rental & Leasing Services,0.010,0.307,hold,6.442,0.330,0.202,2024-03-31
11,"D.R. Horton, Inc.",Residential Construction,0.008,0.176,buy,8.344,0.221,0.244,2024-03-31
5,Lam Research Corporation,Semiconductor Equipment & Materials,0.008,0.311,buy,43.385,0.439,0.187,2024-03-31
7,"Applied Materials, Inc.",Semiconductor Equipment & Materials,0.007,0.306,buy,9.127,0.452,0.093,2024-04-28


In [ ]:
file = "/content/drive/MyDrive/Investimenti/finanza quant/simple_kpis.csv"

In [ ]:
esistente = pd.read_csv(file, usecols=range(1, len(pd.read_csv(file, nrows=1).columns)))

In [ ]:
esistente['mostRecentQuarter'] = pd.to_datetime(esistente['mostRecentQuarter'])
nuovo['mostRecentQuarter'] = pd.to_datetime(nuovo['mostRecentQuarter'])

esistente['shortName'] = esistente['shortName'].astype(str)
nuovo['shortName'] = nuovo['shortName'].astype(str)

esistente['industry'] = esistente['industry'].astype(str)
nuovo['industry'] = nuovo['industry'].astype(str)


In [ ]:
appeso = pd.concat([esistente,nuovo]).drop_duplicates(subset=["shortName","mostRecentQuarter"]).sort_values(by=['shortName', 'mostRecentQuarter'])


In [ ]:
appeso.to_csv(file)
